## Copy Scripts to Current Directory

In [ ]:
!rm -r tcvae/
!rm -r rg_sound_generation/
!rm -r checkpoints/
!rm *

In [ ]:
!git clone https://github.com/TheSoundOfAIOSR/rg_sound_generation.git
!mv /content/rg_sound_generation/timbre_conditioned_vae/* .

In [ ]:
!pip install -q -r requirements_colab.txt

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

## Import Scripts and Setup Config

In [ ]:
from tcvae import localconfig, train

data_handler_type = "data_handler"
conf = localconfig.LocalConfig(data_handler_type)

In [ ]:
# %%writefile conf.txt 
conf.dataset_dir = "/content/drive/MyDrive/the_sound_of_ai/new_data"
conf.checkpoints_dir = "/content/drive/MyDrive/the_sound_of_ai/new_data/checkpoints"
conf.pretrained_model_path = "/content/drive/MyDrive/the_sound_of_ai/new_data/checkpoints/mt_simple_5_run2.h5"
conf.best_loss = 0.007439

# Architecture
conf.use_embeddings = False
conf.use_encoder = True
conf.simple_encoder = True
conf.simple_decoder = True
conf.using_categorical = False
conf.use_max_pool = False
conf.is_variational = False
conf.use_note_number = True
conf.use_velocity = True
conf.use_heuristics = True
conf.latent_dim = 5
conf.print_model_summary = False

# Outputs
conf.mt_outputs["f0_shifts"]["enabled"] = True
conf.mt_outputs["h_freq_shifts"]["enabled"] = True
conf.mt_outputs["mag_env"]["enabled"] = True
conf.mt_outputs["h_mag_dist"]["enabled"] = True
conf.mt_outputs["h_phase_diff"]["enabled"] = False
conf.data_handler.losses_weights["f0_loss"] = 1.0
conf.data_handler.losses_weights["h_freq_shifts_loss"] = 1.0
conf.data_handler.losses_weights["mag_env_loss"] = 1.0
conf.data_handler.losses_weights["h_mag_dist_loss"] = 1.0
conf.data_handler.losses_weights["h_mag_loss"] = 1.0
conf.data_handler.losses_weights["h_phase_diff_loss"] = 0.0
conf.data_handler.use_phase = True

# Training
conf.batch_size = 16
conf.learning_rate = 0.00015
conf.lr_factor = 0.5
conf.lr_plateau = 3
conf.model_name = "mt_simple_5"
conf.early_stopping = 12

conf.save_config()

## Freeze Everything Except 1 Output

In [ ]:
f0_shifts_layers = ["permute", "dense_1", "permute_1", "dense_2", "f0_shifts_decoder_conv_0", "f0_shifts_decoder_bn_0", "f0_shifts_decoder_act_0",
                    "add_9", "f0_shifts_decoder_conv_1", "f0_shifts_decoder_bn_1", "f0_shifts_decoder_act_1", "add_10", 
                    "f0_shifts_decoder_conv_2", "f0_shifts_decoder_bn_2", "f0_shifts_decoder_act_2", "add_11",
                    "f0_shifts_decoder_conv_3", "f0_shifts_decoder_bn_3", "f0_shifts_decoder_act_3", "add_12",
                    "conv2d_15", "lambda_1", "dense_3", "layer_normalization_1", "dense_4", "add_13", "layer_normalization_2",
                    "dense_5", "add_14", "layer_normalization_3", "dense_6"]

## Train

In [ ]:
# If training just an output submodel: (Only works when pretrained model path is specified)
# train.train(conf, f0_shifts_layers)

# If training from scratch
train.train(conf)